**Form4 filings Part 1**

In the following code cell, provide the code for your function.

In [4]:
# Code cell for part 1 - do not delete or modify this line
import os
import pandas as pd
import xml.etree.ElementTree as ET

def CreateDataFrame(basefolder):
    data = []

    for folder_name in os.listdir(basefolder):
        folder_path = os.path.join(basefolder, folder_name)
        for file_name in os.listdir(folder_path):
            _, file_extension = os.path.splitext(file_name)
            
            file_path = os.path.join(folder_path, file_name)
            
            with open(file_path, 'r') as file:
                content = file.read()
                start_tag = '<ownershipDocument>'
                end_tag = '</ownershipDocument>'
                start_index = content.find(start_tag)
                end_index = content.find(end_tag)

                if start_index != -1 and end_index != -1:
                    doc = content[start_index:end_index + len(end_tag)]
                    root = ET.fromstring(doc)
                    
                    # If a filing does not have a single owner (rptOwnerName), you should completely ignore that filing
                    owner_elements = root.findall('.//reportingOwnerId/rptOwnerName')
                    if len(owner_elements) > 1:
                        continue  
                    owner_name = owner_elements[0].text
                    
                    is_dir = root.find('.//reportingOwnerRelationship/isDirector').text if root.find('.//reportingOwnerRelationship/isDirector') is not None else 0
                    is_off = root.find('.//reportingOwnerRelationship/isOfficer').text if root.find('.//reportingOwnerRelationship/isOfficer') is not None else 0
                    is_ten = root.find('.//reportingOwnerRelationship/isTenPercentOwner').text if root.find('.//reportingOwnerRelationship/isTenPercentOwner') is not None else 0

                    for transaction in root.findall('.//nonDerivativeTransaction'):
                        sec_title = transaction.find('.//securityTitle/value')
                        sec_title = sec_title.text if sec_title is not None else ''
                        trans_date = transaction.find('.//transactionDate/value')
                        trans_date = trans_date.text if trans_date is not None else ''
                        shares = transaction.find('.//transactionShares/value')
                        shares = shares.text if shares is not None else 0
                        pps = transaction.find('.//transactionPricePerShare/value')
                        pps = pps.text if pps is not None else 0.0
                        adcode = transaction.find('.//transactionAcquiredDisposedCode/value')
                        adcode = adcode.text if adcode is not None else ''
                        shares_after = transaction.find('.//postTransactionAmounts/sharesOwnedFollowingTransaction/value')
                        shares_after = shares_after.text if shares_after is not None else 0
                        di_owner = transaction.find('.//ownershipNature/directOrIndirectOwnership/value')
                        di_owner = di_owner.text if di_owner is not None else ''
                        
                        data.append({
                            'Folder': folder_name,
                            'OwnerName': owner_name,
                            'IsDir': is_dir,
                            'IsOff': is_off,
                            'IsTen': is_ten,
                            'SecTitle': sec_title,
                            'TransDate': trans_date,
                            'Shares': shares,
                            'PPS': pps,
                            'ADCode': adcode,
                            'SharesAfter': shares_after,
                            'DIOwner': di_owner
                        })

    df = pd.DataFrame(data)
    return df

In [5]:
df_PFE = CreateDataFrame('C:\\AMS\\PFE\\4')

In [7]:
df_PFE.to_csv('C:\\AMS\\jupyter_practice\\Final Project\\de_pfe.csv', index=False)

In [6]:
df_PFE.info
#df_pfe.head(10)

<bound method DataFrame.info of                     Folder             OwnerName IsDir IsOff IsTen  \
0     0000078003-03-000157       MITCHELL JOHN W     0     1     0   
1     0000078003-03-000183  CANGIALOSI LORETTA V     0     1     0   
2     0000078003-03-000183  CANGIALOSI LORETTA V     0     1     0   
3     0000078003-03-000227          CORR PETER B     0     1     0   
4     0000078003-03-000227          CORR PETER B     0     1     0   
...                    ...                   ...   ...   ...   ...   
2416  0001225208-23-004094    DAMICO JENNIFER B.     0     1     0   
2417  0001225208-23-004397           Pao William     0     1     0   
2418  0001225208-23-005604        Gottlieb Scott     1     0     0   
2419  0001225208-23-005604        Gottlieb Scott     1     0     0   
2420  0001225208-23-006419        Denton David M     0     1     0   

          SecTitle   TransDate      Shares      PPS ADCode SharesAfter DIOwner  
0     Common Stock  2003-07-30       19044    